In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision.models import resnet101, ResNet101_Weights

In [44]:
#Load optical flow data from the NPZ file
optical_flow_data = np.load("C:/Users/andre/OneDrive/Documents/GitHub/Brainvision_Project/Motion_files/batch_0_Optical_Flow_Stack.npz", allow_pickle=True)

print(optical_flow_data.keys())

# Assuming the optical flow data is stored in a variable named "flow_data"
flow_data = optical_flow_data["Data"]

# Print the shape of the flow_data array to see its dimensions

first_flow_stack = flow_data[0]
tensor_input = torch.tensor(first_flow_stack, dtype=torch.float32)
print("Shape of first_video:", first_flow_stack.shape)

KeysView(NpzFile 'C:/Users/andre/OneDrive/Documents/GitHub/Brainvision_Project/Motion_files/batch_0_Optical_Flow_Stack.npz' with keys: File_ID, Data)
Shape of first_video: (1, 16, 224, 224)


In [39]:
class ModifiedResNet101(nn.Module):
    def __init__(self, num_classes=101, input_channels=16):
        super(ModifiedResNet101, self).__init__()
        # Load a pre-trained resnet101 model with the new weights parameter
        original_model = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)

        # Modify the first convolutional layer to accept 16-channel input
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False),
            original_model.bn1,
            original_model.relu,
            original_model.maxpool,
        )

        # Use the rest of the ResNet101 layers
        self.layer1 = original_model.layer1
        self.layer2 = original_model.layer2
        self.layer3 = original_model.layer3
        self.layer4 = original_model.layer4
        self.avgpool = original_model.avgpool

        # Replace the final fully connected layer
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Example usage
model = ModifiedResNet101(num_classes=101, input_channels=16)

In [35]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

CUDA Available: False


In [37]:
# Move the model to GPU if CUDA is available
if cuda_available:
    model = model.to('cuda')

In [45]:
# Example of using the model with a dummy input
dummy_input = tensor_input  
# Move the data to GPU if CUDA is available
if cuda_available:
    dummy_input = dummy_input.to('cuda')

# Forward pass
output = model(dummy_input)
print(output)

tensor([[ 0.2603, -0.3102, -0.0202, -0.7101, -0.1261, -0.0641,  0.3179, -0.2516,
         -0.4431, -0.1020,  0.0798,  0.0928, -0.0255, -0.0878,  0.4197,  0.2188,
          0.3150, -0.1335,  0.3236, -0.1151,  0.1148,  0.2013,  0.2850,  0.4231,
          0.2522, -0.2313, -0.1374,  0.2470, -0.2998,  0.3614, -0.1070, -0.4153,
          0.4068,  0.4375, -0.0156,  0.1833, -1.0190, -0.1713, -0.0254, -0.0605,
         -0.0642, -0.2885,  0.0724,  0.0874, -0.2186, -0.1717, -0.2342, -0.1904,
         -0.2229, -0.1224, -0.6913, -0.5461, -0.5211,  0.3701, -0.2457,  0.0075,
         -0.0931, -0.2141,  0.2392,  0.2626, -0.5643, -0.0110,  0.0423,  0.2375,
         -0.4816,  0.1227, -0.2379,  0.1232,  0.5849,  0.0737, -0.0916, -0.1800,
         -0.2122,  0.2754,  0.4036,  0.2834,  0.0301, -0.0661,  0.0038,  0.1998,
         -0.0376, -0.2857, -0.2048,  0.0101, -0.2800, -0.1542, -0.1200,  0.4165,
         -0.0175, -0.3150, -0.0468, -0.1390, -0.6246,  0.0685,  0.2680,  0.4889,
          0.2042, -0.1394, -